In [3]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import pennylane as qml
import cv2
import matplotlib.pyplot as plt

# Define file paths for the images
image_paths = [
    "/content/cat_10.jpeg", "/content/cat_11.jpeg", "/content/dog_2.jpeg",
    "/content/dog_3.jpeg", "/content/dog_4.jpeg", "/content/dog_5.jpeg", "/content/dog_6.jpeg",
    "/content/cat_7.jpeg", "/content/cat_8.jpeg", "/content/cat_9.jpeg"
]

num_qubits = 8  # We can adjust based on how many features we want to encode
dev = qml.device("default.qubit", wires=num_qubits)

def load_and_preprocess_images(image_paths):
    """Load images, resize them to 224x224, and normalize pixel values."""
    images = []
    for image_path in image_paths:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale (1 channel)
        image_resized = cv2.resize(image, (224, 224))  # Resize to 224x224
        image_normalized = image_resized / 255.0  # Normalize pixel values to range [0, 1]
        images.append(image_normalized)
    return np.array(images)

# Load and preprocess the images
images = load_and_preprocess_images(image_paths)
print(f"Loaded and processed {len(images)} images.")

# Convert images into quantum states using basis encoding
def basis_encoding(image_features):
    """Encodes image features into a quantum state using basis encoding."""

    # Convert continuous values to binary (Threshold at 0.5)
    binary_features = (image_features.flatten() > 0.5).astype(int)  # Flatten to 1D array and threshold

    @qml.qnode(dev)
    def circuit():
        """Quantum circuit for basis encoding."""
        for i in range(num_qubits):
            if binary_features[i]:  # If the feature is 1, apply an X gate
                qml.PauliX(wires=i)
        return qml.state()

    return circuit()

# Process all images into quantum states
quantum_states = []
for image in images:
    quantum_state = basis_encoding(image)
    quantum_states.append(quantum_state)
    print(f"Encoded image into quantum state.")

# Convert quantum states to numpy array for further processing
quantum_states = np.array(quantum_states)

# Quantum Fourier Transform (QFT) to apply on quantum states
def qft(wires):
    """Applies Quantum Fourier Transform (QFT) on the given qubits."""
    n = len(wires)

    for i in range(n):
        qml.Hadamard(wires=i)
        for j in range(i+1, n):
            qml.ControlledPhaseShift(np.pi / 2**(j - i), wires=[j, i])

    # Reverse qubit order (QFT convention)
    for i in range(n // 2):
        qml.SWAP(wires=[i, n - i - 1])

@qml.qnode(dev)
def apply_qft(state):
    """Quantum circuit to apply QFT on an input quantum state."""
    qml.StatePrep(state, wires=range(num_qubits))  # Load state
    qft(wires=range(num_qubits))  # Apply QFT
    return qml.state()

# Apply QFT on each quantum-encoded image
qft_transformed_states = []
for state in quantum_states:
    transformed_state = apply_qft(state)
    qft_transformed_states.append(transformed_state)

# Convert to numpy array
qft_transformed_states = np.array(qft_transformed_states)


# You can also save the quantum states after QFT transformation for further analysis if needed
np.save("/content/qft_transformed_images.npy", qft_transformed_states)
print("QFT-transformed states saved successfully.")


Loaded and processed 10 images.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
QFT-transformed states saved successfully.


In [5]:
import numpy as np
import pennylane as qml
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

# Define file paths for the images
image_paths = [
    "/content/cat_10.jpeg", "/content/cat_11.jpeg", "/content/dog_2.jpeg",
    "/content/dog_3.jpeg", "/content/dog_4.jpeg", "/content/dog_5.jpeg", "/content/dog_6.jpeg",
    "/content/cat_7.jpeg", "/content/cat_8.jpeg", "/content/cat_9.jpeg"
]

# True labels for the images (replace with your actual true labels)
true_labels = np.array([1,1, 0,0,0,0,0,1,1,1])  # Example labels

# Quantum device setup: Adjust the number of qubits depending on how you want to encode features
num_qubits = 8  # Adjust based on how many features you want to encode
dev = qml.device("default.qubit", wires=num_qubits)

def load_and_preprocess_images(image_paths):
    """Load images, resize them to 224x224, and normalize pixel values."""
    images = []
    for image_path in image_paths:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale (1 channel)
        image_resized = cv2.resize(image, (224, 224))  # Resize to 224x224
        image_normalized = image_resized / 255.0  # Normalize pixel values to range [0, 1]
        images.append(image_normalized)
    return np.array(images)

# Load and preprocess the images
images = load_and_preprocess_images(image_paths)
print(f"Loaded and processed {len(images)} images.")

# Convert images into quantum states using basis encoding
def basis_encoding(image_features):
    """Encodes image features into a quantum state using basis encoding."""

    # Convert continuous values to binary (Threshold at 0.5)
    binary_features = (image_features.flatten() > 0.5).astype(int)  # Flatten to 1D array and threshold

    @qml.qnode(dev)
    def circuit():
        """Quantum circuit for basis encoding."""
        for i in range(num_qubits):
            if binary_features[i]:  # If the feature is 1, apply an X gate
                qml.PauliX(wires=i)
        return qml.state()

    return circuit()

# Process all images into quantum states
quantum_states = []
for image in images:
    quantum_state = basis_encoding(image)
    quantum_states.append(quantum_state)
    print(f"Encoded image into quantum state.")

# Convert quantum states to numpy array for further processing
quantum_states = np.array(quantum_states)

# Quantum Fourier Transform (QFT) to apply on quantum states
def qft(wires):
    """Applies Quantum Fourier Transform (QFT) on the given qubits."""
    n = len(wires)

    for i in range(n):
        qml.Hadamard(wires=i)
        for j in range(i+1, n):
            qml.ControlledPhaseShift(np.pi / 2**(j - i), wires=[j, i])

    # Reverse qubit order (QFT convention)
    for i in range(n // 2):
        qml.SWAP(wires=[i, n - i - 1])

@qml.qnode(dev)
def apply_qft(state):
    """Quantum circuit to apply QFT on an input quantum state."""
    qml.StatePrep(state, wires=range(num_qubits))  # Load state
    qft(wires=range(num_qubits))  # Apply QFT
    return qml.state()

# Apply QFT on each quantum-encoded image
qft_transformed_states = []
for state in quantum_states:
    transformed_state = apply_qft(state)
    qft_transformed_states.append(transformed_state)

# Convert to numpy array
qft_transformed_states = np.array(qft_transformed_states)

# Flatten the QFT states for classification (we'll use a classical classifier)
qft_flattened = np.abs(qft_transformed_states).reshape(len(qft_transformed_states), -1)

# Train a classical classifier (e.g., K-Nearest Neighbors)
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(qft_flattened, true_labels)

# Predict on the same set (just for demonstration)
predicted_labels = clf.predict(qft_flattened)


Loaded and processed 10 images.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.
Encoded image into quantum state.


In [10]:
import numpy as np
import pennylane as qml
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression  # Example classifier for prediction
image_paths = [
    "/content/cat_10.jpeg", "/content/cat_11.jpeg", "/content/dog_2.jpeg",
    "/content/dog_3.jpeg", "/content/dog_4.jpeg", "/content/dog_5.jpeg", "/content/dog_6.jpeg",
    "/content/cat_7.jpeg", "/content/cat_8.jpeg", "/content/cat_9.jpeg"
]


# Quantum device setup
num_qubits = 8  # Set the number of qubits based on the feature size
dev = qml.device("default.qubit", wires=num_qubits)

# Function to perform Amplitude Encoding
def amplitude_encoding(image_path):
    """Encode image into quantum state using Amplitude Encoding."""
    # Load and process the image
    img = Image.open(image_path).convert("L")  # Convert image to grayscale
    img = np.array(img)  # Convert to numpy array
    img = img.flatten()  # Flatten the image to 1D array
    img = img / np.linalg.norm(img)  # Normalize the values to [0,1] range

    # Check if the image has more pixels than qubits
    if len(img) > num_qubits:
        img = img[:num_qubits]  # Truncate to fit the number of qubits
    elif len(img) < num_qubits:
        img = np.pad(img, (0, num_qubits - len(img)), 'constant')  # Pad with zeros if needed

    @qml.qnode(dev)
    def circuit():
        """Quantum circuit for amplitude encoding."""
        # Apply the image data as amplitudes for the quantum state
        for i in range(num_qubits):
            qml.RX(2 * np.arccos(img[i]), wires=i)  # Use RX gate to encode image values

        return qml.state()

    # Execute the circuit to get the quantum state
    return circuit()

# Process and encode all images
quantum_states = []
for image_path in image_paths:
    quantum_state = amplitude_encoding(image_path)
    quantum_states.append(quantum_state)
    print(f"Encoded {image_path} into quantum state.")

# Save the quantum states as .npy files
np.save("/content/amplitude_encoded_images.npy", quantum_states)
print("Quantum states saved successfully.")

# Extract real and imaginary parts of the quantum states for classification
quantum_states_flat = []
for state in quantum_states:
    # Flatten the state and take real and imaginary parts
    real_part = np.real(state)
    imag_part = np.imag(state)
    quantum_states_flat.append(np.concatenate([real_part, imag_part]))

# Convert to numpy array for sklearn
quantum_states_flat = np.array(quantum_states_flat)

# Train a simple classifier (e.g., Logistic Regression) to classify quantum states
clf = LogisticRegression(max_iter=1000)
clf.fit(quantum_states_flat, true_labels)

# Predict the labels
predicted_labels = clf.predict(quantum_states_flat)



Encoded /content/cat_10.jpeg into quantum state.
Encoded /content/cat_11.jpeg into quantum state.
Encoded /content/dog_2.jpeg into quantum state.
Encoded /content/dog_3.jpeg into quantum state.
Encoded /content/dog_4.jpeg into quantum state.
Encoded /content/dog_5.jpeg into quantum state.
Encoded /content/dog_6.jpeg into quantum state.
Encoded /content/cat_7.jpeg into quantum state.
Encoded /content/cat_8.jpeg into quantum state.
Encoded /content/cat_9.jpeg into quantum state.
Quantum states saved successfully.
